In [1]:
%load_ext autoreload
%autoreload 2

import weave
from loguru import logger
import os
import random
from mistralai.client import MistralClient
from mistralai.models.jobs import WandbIntegrationIn, TrainingParameters

from mistral_fine_tuning.utils import read_fine_tuning_file
from mistral_fine_tuning.reformat import reformat_jsonl

from dotenv import load_dotenv
load_dotenv()


True

In [2]:
df = read_fine_tuning_file('../data/interim/quality_jokes_fine_tuning.jsonl')

In [3]:
random.seed(100)

def extract_random_element(row):
    return random.choice(row['keywords'].split(',')).replace('[', '').replace(']', '')

df['one_keyword'] = df.apply(extract_random_element, axis=1)

In [4]:
df.head()

,text,keywords,one_keyword
0,"Oye, fjate que llega un indio al mdico y qu pa...","['indio', 'mdico', 'Toro Sentado', 'enfermo', ...",'mdico'
1,y llega el tema del medio que le dice doctor m...,"['luna de viernes', 'seora', 'tres pechos', 'd...",'doctor'
2,"Oye, llega un tipo a una tienda de deportes to...","['desnudo', 'tienda de deportes', 'zapatillas'...",'descuento'
3,Y llega la Caperucita Roja a ver a la abuelita...,"['Caperucita Roja', 'abuelita', 'ojos grandes'...",'abuelita'
4,"La situación, la crisis iniciática no tiene má...","['crisis', 'agua', 'playa', 'edificio en const...",'edificio en construcción'


In [6]:
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)
len(df_train), len(df_eval)

(2118, 111)

In [ ]:
weave.init("mistral_webinar")
ds_train = weave.Dataset(name="ds_train", rows=df_train)
ds_eval = weave.Dataset(name="ds_eval", rows=df_eval)